# Function Calling

*__NOTE:__
For instructions on running the Jupyter Notebook that contains the labs see instructions here: <https://github.com/retaildevcrews/OpenAI-Labs>*

This lab shows examples of leveraging response_format property to return results structured as a json object.  This feature is available in the newest previews of gpt-4 and gpt-35.

<https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/json-mode>



In [ ]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os

import openai
from dotenv import load_dotenv

load_dotenv()


In [ ]:
# Load config values

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv("OPENAI_API_BASE")
    
# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# Setting up the deployment name
chatgpt_model_name = os.getenv("CHATGPT_MODEL_GPT4")

# Currently Chat Completions API have the following versions available: 2023-03-15-preview
openai.api_version = os.getenv("OPENAI_API_VERSION")


## 1.0 Define Request to OpenAI API
For these labs we are using the Azure OpenAI Completion API, this is different than the Chat completion API.  Information about its usage can be found here:
(https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#completions)
The function defined below takes prompt and settings parameters sends the request to the endpoint and prints the text element from the result.


In [ ]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_prompt(prompt, temperature=0.5, top_p=0.5, system_message="you are an assistant that helps people with their math homework."):
    messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content":prompt}
    ]
    functions= [  
        {
            "name": "add",
            "description": "Adds two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "The first number"
                    },
                    "b": {
                        "type": "number",
                        "description": "The second number"
                    }
                },
                "required": ["a", "b"]
            }   
        },
        {
            "name": "divide",
            "description": "Divides two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "The first number"
                    },
                    "b": {
                        "type": "number",
                        "description": "The second number"
                    }
                },
                "required": ["a", "b"]
            }   
        },
        {
            "name": "subtract",
            "description": "Subtracts two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "The first number"
                    },
                    "b": {
                        "type": "number",
                        "description": "The second number"
                    }
                },
                "required": ["a", "b"]
            }   
        },
        {
            "name": "multiply",
            "description": "Multiplies two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "The first number"
                    },
                    "b": {
                        "type": "number",
                        "description": "The second number"
                    }
                },
                "required": ["a", "b"]
            }   
        }
    ]  

    response = openai.chat.completions.create(
        messages=messages,
        model=chatgpt_model_name,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0,
        functions=functions,
        function_call="auto"
    )
    response_message=response.choices[0].message
    print("Settings:")
    print("temperature: "+ str(temperature) + " top_p: " + str(top_p))
    print("Prompt: " + prompt)
    print("Response: ")
    print(str(response_message))
    print("----")
    return response_message


## 2.0 Use FunctionCall Response to call a functiona

In [ ]:
import json

def add(a,b):
    return a+b
def subtract(a,b):
    return a-b
def multiply(a,b):  
    return a*b
def divide(a,b):        
    return a/b

def call_function(response_message):
    # Check if the model wants to call a function
    if response_message.function_call is not None:
        # Call the function. The JSON response may not always be valid so make sure to handle errors
        function_name = response_message.function_call.name
        function_args = json.loads(response_message.function_call.arguments)

        available_functions = {
                "add": add,
                "subtract": subtract,
                "multiply": multiply,
                "divide": divide
        }
        function_to_call = available_functions[function_name] 
        function_response = function_to_call(**function_args)
        print(f"Called function: {function_name} with arguments: {function_args} and received response: {function_response}")
        
    else:
        print("Response message did not contain function call: "+response_message)


## 3.0 Send Prompt that results in a function being called

In [ ]:
prompt =  """
What is 4+5
"""
call_function(send_prompt(prompt=prompt))

prompt =  """
divide 90 by 3
"""
call_function(send_prompt(prompt=prompt))
